In [2]:
import pandas as pd
years = [18, 19, 20, 21, 22, 23, 24, 25]
headers = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "new_build_flag",
    "tenure_type",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status"
]
drop = [
    "transaction_id",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "ppd_category_type",
    "record_status"
]
dfs = {}
for year in years:
    url = f"http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-20{year}.csv"
    df = pd.read_csv(url)
    df.columns = headers
    df = df.drop(columns=drop)
    dfs[f"df20{year}"] = df
    print(f"{year} done")
dfs

18 done
19 done
20 done
21 done
22 done


C:\Users\lenovo\AppData\Local\Temp\ipykernel_11548\2815900718.py:33: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


23 done
24 done
25 done


{'df2018':           price  date_of_transfer  postcode property_type new_build_flag  \
 0        253500  2018-09-24 00:00    M6 8GQ             D              N   
 1        231950  2018-09-28 00:00   WA3 2UE             D              Y   
 2        112500  2018-08-29 00:00   OL6 6RJ             S              N   
 3        184995  2018-06-15 00:00   M46 0TW             S              Y   
 4        214995  2018-09-28 00:00   M28 3XS             D              Y   
 ...         ...               ...       ...           ...            ...   
 1037248  100000  2018-11-16 00:00   DH4 7HW             T              N   
 1037249  334950  2018-11-30 00:00  NE31 1BF             D              Y   
 1037250  106010  2018-12-12 00:00   NE2 1NY             F              N   
 1037251  144995  2018-07-26 00:00  NE27 0JA             T              Y   
 1037252  334950  2018-07-13 00:00  NE15 9BW             D              Y   
 
         tenure_type            town_city             district  

In [3]:
df_full = pd.concat(dfs)

In [4]:
df_full['Price (Thousands)'] = df_full['price'] / 1000
df_full.drop(columns=['price'], inplace=True)
df_full.describe()

,Price (Thousands)
count,7.219742e+06
mean,3.818475e+02
std,1.711452e+03
min,1.000000e-03
25%,1.650000e+02
50%,2.600000e+02
75%,4.000000e+02
max,9.000000e+05


In [5]:
df_full.isna().sum()

date_of_transfer         0
postcode             24503
property_type            0
new_build_flag           0
tenure_type              0
town_city                0
district                 0
county                   0
Price (Thousands)        0
dtype: int64

In [6]:
df_clean = df_full.dropna(axis=0)
df_clean.describe()

,Price (Thousands)
count,7.195239e+06
mean,3.794882e+02
std,1.660432e+03
min,1.000000e-03
25%,1.650000e+02
50%,2.600000e+02
75%,4.000000e+02
max,9.000000e+05


In [7]:
df_clean.head()

date_of_transfer postcode property_type new_build_flag tenure_type  \
df2018 0  2018-09-24 00:00   M6 8GQ             D              N           F   
       1  2018-09-28 00:00  WA3 2UE             D              Y           F   
       2  2018-08-29 00:00  OL6 6RJ             S              N           F   
       3  2018-06-15 00:00  M46 0TW             S              Y           F   
       4  2018-09-28 00:00  M28 3XS             D              Y           L   

                  town_city  district              county  Price (Thousands)  
df2018 0            SALFORD   SALFORD  GREATER MANCHESTER            253.500  
       1         WARRINGTON     WIGAN  GREATER MANCHESTER            231.950  
       2  ASHTON-UNDER-LYNE  TAMESIDE  GREATER MANCHESTER            112.500  
       3         MANCHESTER     WIGAN  GREATER MANCHESTER            184.995  
       4         MANCHESTER   SALFORD  GREATER MANCHESTER            214.995

In [8]:
df_binary = pd.get_dummies(df_clean, dtype=int, columns=['tenure_type','new_build_flag', 'property_type'])
df_binary.rename(columns={'date_of_transfer':'Transfer Date','town_city':'Town/City','district':'District','county':'County','tenure_type_F':'Freehold Tenure','tenure_type_L':'Leasehold Tenure','new_build_flag_N':'Old Build','new_build_flag_Y':'New Build','property_type_D':'Detached', 'property_type_F':'Flat', 'property_type_O':'Other Property Type','property_type_S':'Semi-detached','property_type_T':'Terraced', 'postcode':'Postcode'}, inplace=True)
df_binary.head()

Transfer Date Postcode          Town/City  District  \
df2018 0  2018-09-24 00:00   M6 8GQ            SALFORD   SALFORD   
       1  2018-09-28 00:00  WA3 2UE         WARRINGTON     WIGAN   
       2  2018-08-29 00:00  OL6 6RJ  ASHTON-UNDER-LYNE  TAMESIDE   
       3  2018-06-15 00:00  M46 0TW         MANCHESTER     WIGAN   
       4  2018-09-28 00:00  M28 3XS         MANCHESTER   SALFORD   

                      County  Price (Thousands)  Freehold Tenure  \
df2018 0  GREATER MANCHESTER            253.500                1   
       1  GREATER MANCHESTER            231.950                1   
       2  GREATER MANCHESTER            112.500                1   
       3  GREATER MANCHESTER            184.995                1   
       4  GREATER MANCHESTER            214.995                0   

          Leasehold Tenure  Old Build  New Build  Detached  Flat  \
df2018 0                 0          1          0         1     0   
       1                 0          0          1         1     0   
       2                 0          1          0         0     0   
       3                 0          0          1         0     0   
       4                 1          0          1         1     0   

          Other Property Type  Semi-detached  Terraced  
df2018 0                    0              0         0  
       1                    0              0         0  
       2                    0              1         0  
       3                    0              1         0  
       4                    0              0         0

In [9]:
df_binary['Transfer Date'] = pd.to_datetime(df_binary['Transfer Date'], errors='coerce')
df_binary['Year'] = df_binary['Transfer Date'].dt.year
df_binary['Month'] = df_binary['Transfer Date'].dt.month
df_binary['Quarter'] = df_binary['Transfer Date'].dt.quarter
df_binary['Day of the Week'] = df_binary['Transfer Date'].dt.dayofweek
df_binary['Transfer Date'] = df_binary['Transfer Date'].astype(str).str[:10]
df_binary.tail()

Transfer Date  Postcode    Town/City                District  \
df2025 252142    2025-01-17   NN4 8SA  NORTHAMPTON   WEST NORTHAMPTONSHIRE   
       252143    2025-01-24  NN16 0SE    KETTERING  NORTH NORTHAMPTONSHIRE   
       252144    2025-01-14  NN10 0NB      RUSHDEN  NORTH NORTHAMPTONSHIRE   
       252145    2025-01-10   NN4 8TE  NORTHAMPTON   WEST NORTHAMPTONSHIRE   
       252146    2025-01-27  NN15 5YL    KETTERING  NORTH NORTHAMPTONSHIRE   

                               County  Price (Thousands)  Freehold Tenure  \
df2025 252142   WEST NORTHAMPTONSHIRE              98.00                0   
       252143  NORTH NORTHAMPTONSHIRE             120.00                0   
       252144  NORTH NORTHAMPTONSHIRE             604.50                1   
       252145   WEST NORTHAMPTONSHIRE             133.65                0   
       252146  NORTH NORTHAMPTONSHIRE             332.00                1   

               Leasehold Tenure  Old Build  New Build  Detached  Flat  \
df2025 252142                 1          1          0         0     1   
       252143                 1          1          0         0     1   
       252144                 0          1          0         0     0   
       252145                 1          1          0         0     1   
       252146                 0          1          0         1     0   

               Other Property Type  Semi-detached  Terraced  Year  Month  \
df2025 252142                    0              0         0  2025      1   
       252143                    0              0         0  2025      1   
       252144                    1              0         0  2025      1   
       252145                    0              0         0  2025      1   
       252146                    0              0         0  2025      1   

               Quarter  Day of the Week  
df2025 252142        1                4  
       252143        1                4  
       252144        1                1  
       252145        1                4  
       252146        1                0

In [10]:
df_binary.shape

(7195239, 19)

In [11]:
print(df_binary.dtypes)
print(df_binary.nunique())

Transfer Date           object
Postcode                object
Town/City               object
District                object
County                  object
Price (Thousands)      float64
Freehold Tenure          int64
Leasehold Tenure         int64
Old Build                int64
New Build                int64
Detached                 int64
Flat                     int64
Other Property Type      int64
Semi-detached            int64
Terraced                 int64
Year                     int32
Month                    int32
Quarter                  int32
Day of the Week          int32
dtype: object
Transfer Date             2731
Postcode               1162297
Town/City                 1150
District                   361
County                     117
Price (Thousands)       143229
Freehold Tenure              2
Leasehold Tenure             2
Old Build                    2
New Build                    2
Detached                     2
Flat                         2
Other Property Type      

In [12]:
df_binary['Postcode Area'] = df_binary['Postcode'].str.extract(r'^([A-Z]+)')
df_binary['Postcode District'] = df_binary['Postcode'].str.extract(r'^([A-Z]+[0-9]+)')
df_binary.head()

Transfer Date Postcode          Town/City  District  \
df2018 0    2018-09-24   M6 8GQ            SALFORD   SALFORD   
       1    2018-09-28  WA3 2UE         WARRINGTON     WIGAN   
       2    2018-08-29  OL6 6RJ  ASHTON-UNDER-LYNE  TAMESIDE   
       3    2018-06-15  M46 0TW         MANCHESTER     WIGAN   
       4    2018-09-28  M28 3XS         MANCHESTER   SALFORD   

                      County  Price (Thousands)  Freehold Tenure  \
df2018 0  GREATER MANCHESTER            253.500                1   
       1  GREATER MANCHESTER            231.950                1   
       2  GREATER MANCHESTER            112.500                1   
       3  GREATER MANCHESTER            184.995                1   
       4  GREATER MANCHESTER            214.995                0   

          Leasehold Tenure  Old Build  New Build  ...  Flat  \
df2018 0                 0          1          0  ...     0   
       1                 0          0          1  ...     0   
       2                 0          1          0  ...     0   
       3                 0          0          1  ...     0   
       4                 1          0          1  ...     0   

          Other Property Type  Semi-detached  Terraced  Year  Month  Quarter  \
df2018 0                    0              0         0  2018      9        3   
       1                    0              0         0  2018      9        3   
       2                    0              1         0  2018      8        3   
       3                    0              1         0  2018      6        2   
       4                    0              0         0  2018      9        3   

          Day of the Week  Postcode Area Postcode District  
df2018 0                0              M                M6  
       1                4             WA               WA3  
       2                2             OL               OL6  
       3                4              M               M46  
       4                4              M               M28  

[5 rows x 21 columns]

In [13]:
df_avg_pc = df_binary.groupby('Postcode Area')['Price (Thousands)'].mean().reset_index()
df_avg_pc.columns = ['Postcode Area', 'Area Code Average Price (Thousands)']
df_avg_county = df_binary.groupby('County')['Price (Thousands)'].mean().reset_index()
df_avg_county.columns = ['County', 'County Average Price (Thousands)']
df_avg_tc = df_binary.groupby('Town/City')['Price (Thousands)'].mean().reset_index()
df_avg_tc.columns = ['Town/City', 'Town/City Average Price (Thousands)']
df_avg_dist = df_binary.groupby('District')['Price (Thousands)'].mean().reset_index()
df_avg_dist.columns = ['District', 'District Average Price (Thousands)']
df_avg_pc.head()

,Postcode Area,Area Code Average Price (Thousands)
0,AL,624.364499
1,B,302.164939
2,BA,397.046740
3,BB,186.553792
4,BD,198.704839


In [14]:
df_merge = df_binary.merge(df_avg_pc, on=['Postcode Area'], how='left')
df_merge = df_merge.merge(df_avg_tc, on=['Town/City'], how='left')
df_merge = df_merge.merge(df_avg_dist, on=['District'], how='left')
df_merge = df_merge.merge(df_avg_county, on=['County'], how='left')
df_merge.head()

,Transfer Date,Postcode,Town/City,District,County,Price (Thousands),Freehold Tenure,Leasehold Tenure,Old Build,New Build,...,Year,Month,Quarter,Day of the Week,Postcode Area,Postcode District,Area Code Average Price (Thousands),Town/City Average Price (Thousands),District Average Price (Thousands),County Average Price (Thousands)
0,2018-09-24,M6 8GQ,SALFORD,SALFORD,GREATER MANCHESTER,253.500,1,0,1,0,...,2018,9,3,0,M,M6,286.373466,261.292260,253.098761,262.010594
1,2018-09-28,WA3 2UE,WARRINGTON,WIGAN,GREATER MANCHESTER,231.950,1,0,0,1,...,2018,9,3,4,WA,WA3,301.647233,289.308785,193.246849,262.010594
2,2018-08-29,OL6 6RJ,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,112.500,1,0,1,0,...,2018,8,3,2,OL,OL6,197.639388,183.803044,196.658531,262.010594
3,2018-06-15,M46 0TW,MANCHESTER,WIGAN,GREATER MANCHESTER,184.995,1,0,0,1,...,2018,6,2,4,M,M46,286.373466,285.381745,193.246849,262.010594
4,2018-09-28,M28 3XS,MANCHESTER,SALFORD,GREATER MANCHESTER,214.995,0,1,0,1,...,2018,9,3,4,M,M28,286.373466,285.381745,253.098761,262.010594


In [15]:
loc_df = pd.read_csv('ONSPD_Online_Latest_Centroids.csv')
loc_df.head()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_11548\1580054128.py:1: DtypeWarning: Columns (34,42,47,55) have mixed types. Specify dtype option on import or set low_memory=False.
  loc_df = pd.read_csv('ONSPD_Online_Latest_Centroids.csv')


,X,Y,OBJECTID,PCD,PCD2,PCDS,DOINTR,DOTERM,OSCTY,CED,...,LEP2,PFA,IMD,CALNCV,ICB,OA21,LSOA21,MSOA21,RUC21IND,GlobalID
0,385386.0,801193.0,1,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606.0,S99999999,S99999999,...,NaN,S23000009,6715,S99999999,S99999999,S00137176,S01013490,S02002516,NaN,b2d74491-a06c-4dd0-be0d-3f3fd5997475
1,385177.0,801314.0,2,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606.0,S99999999,S99999999,...,NaN,S23000009,6715,S99999999,S99999999,S00137176,S01013490,S02002516,NaN,83e9e27a-724a-4020-9cf0-d87eff5872e8
2,385053.0,801092.0,3,AB1 0AD,AB1 0AD,AB1 0AD,198001,199606.0,S99999999,S99999999,...,NaN,S23000009,6715,S99999999,S99999999,S00137176,S01013490,S02002516,NaN,994eb987-1286-4c7f-b16b-87d03f6b0dec
3,384600.0,799300.0,4,AB1 0AE,AB1 0AE,AB1 0AE,199402,199606.0,S99999999,S99999999,...,NaN,S23000009,5069,S99999999,S99999999,S00138891,S01013856,S02002577,NaN,fd4a2772-2414-4d69-ba99-311e90df23d5
4,384460.0,800660.0,5,AB1 0AF,AB1 0AF,AB1 0AF,199012,199207.0,S99999999,S99999999,...,NaN,S23000009,6253,S99999999,S99999999,S00137241,S01013487,S02002515,NaN,270fcb01-bcda-42fc-8c38-6fdea1c3d1a3


In [16]:
loc_df.columns

Index(['X', 'Y', 'OBJECTID', 'PCD', 'PCD2', 'PCDS', 'DOINTR', 'DOTERM',
       'OSCTY', 'CED', 'OSLAUA', 'OSWARD', 'PARISH', 'USERTYPE', 'OSEAST1M',
       'OSNRTH1M', 'OSGRDIND', 'OSHLTHAU', 'NHSER', 'CTRY', 'RGN', 'STREG',
       'PCON', 'EER', 'TECLEC', 'TTWA', 'PCT', 'ITL', 'STATSWARD', 'OA01',
       'CASWARD', 'NPARK', 'LSOA01', 'MSOA01', 'UR01IND', 'OAC01', 'OA11',
       'LSOA11', 'MSOA11', 'WZ11', 'SICBL', 'BUA24', 'RU11IND', 'OAC11', 'LAT',
       'LONG', 'LEP1', 'LEP2', 'PFA', 'IMD', 'CALNCV', 'ICB', 'OA21', 'LSOA21',
       'MSOA21', 'RUC21IND', 'GlobalID'],
      dtype='object')

In [17]:
loc_df_drop = loc_df[['LAT','LONG','PCDS','OSEAST1M','OSNRTH1M']]
loc_df_drop.rename(columns={'PCDS':'Postcode'}, inplace=True)
loc_df_drop.head()


C:\Users\lenovo\AppData\Local\Temp\ipykernel_11548\1127414209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc_df_drop.rename(columns={'PCDS':'Postcode'}, inplace=True)


,LAT,LONG,Postcode,OSEAST1M,OSNRTH1M
0,57.10146,-2.242858,AB1 0AA,385386.0,801193.0
1,57.10254,-2.246315,AB1 0AB,385177.0,801314.0
2,57.10054,-2.248349,AB1 0AD,385053.0,801092.0
3,57.08443,-2.255714,AB1 0AE,384600.0,799300.0
4,57.09664,-2.258109,AB1 0AF,384460.0,800660.0


In [18]:
loc_house_df = df_merge.merge(loc_df_drop, on=['Postcode'], how='left')
loc_house_df.head()


,Transfer Date,Postcode,Town/City,District,County,Price (Thousands),Freehold Tenure,Leasehold Tenure,Old Build,New Build,...,Postcode Area,Postcode District,Area Code Average Price (Thousands),Town/City Average Price (Thousands),District Average Price (Thousands),County Average Price (Thousands),LAT,LONG,OSEAST1M,OSNRTH1M
0,2018-09-24,M6 8GQ,SALFORD,SALFORD,GREATER MANCHESTER,253.500,1,0,1,0,...,M,M6,286.373466,261.292260,253.098761,262.010594,53.49295,-2.311376,379439.0,399626.0
1,2018-09-28,WA3 2UE,WARRINGTON,WIGAN,GREATER MANCHESTER,231.950,1,0,0,1,...,WA,WA3,301.647233,289.308785,193.246849,262.010594,53.46955,-2.583977,361332.0,397136.0
2,2018-08-29,OL6 6RJ,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,112.500,1,0,1,0,...,OL,OL6,197.639388,183.803044,196.658531,262.010594,53.48984,-2.074338,395165.0,399237.0
3,2018-06-15,M46 0TW,MANCHESTER,WIGAN,GREATER MANCHESTER,184.995,1,0,0,1,...,M,M46,286.373466,285.381745,193.246849,262.010594,53.52589,-2.495181,367270.0,403360.0
4,2018-09-28,M28 3XS,MANCHESTER,SALFORD,GREATER MANCHESTER,214.995,0,1,0,1,...,M,M28,286.373466,285.381745,253.098761,262.010594,53.51677,-2.411523,372810.0,402310.0


In [19]:
loc_house_df.isna().sum()

Transfer Date                            0
Postcode                                 0
Town/City                                0
District                                 0
County                                   0
Price (Thousands)                        0
Freehold Tenure                          0
Leasehold Tenure                         0
Old Build                                0
New Build                                0
Detached                                 0
Flat                                     0
Other Property Type                      0
Semi-detached                            0
Terraced                                 0
Year                                     0
Month                                    0
Quarter                                  0
Day of the Week                          0
Postcode Area                            0
Postcode District                        0
Area Code Average Price (Thousands)      0
Town/City Average Price (Thousands)      0
District Av

In [20]:
loc_house_df.dropna(axis=0)

,Transfer Date,Postcode,Town/City,District,County,Price (Thousands),Freehold Tenure,Leasehold Tenure,Old Build,New Build,...,Postcode Area,Postcode District,Area Code Average Price (Thousands),Town/City Average Price (Thousands),District Average Price (Thousands),County Average Price (Thousands),LAT,LONG,OSEAST1M,OSNRTH1M
0,2018-09-24,M6 8GQ,SALFORD,SALFORD,GREATER MANCHESTER,253.500,1,0,1,0,...,M,M6,286.373466,261.292260,253.098761,262.010594,53.49295,-2.311376,379439.0,399626.0
1,2018-09-28,WA3 2UE,WARRINGTON,WIGAN,GREATER MANCHESTER,231.950,1,0,0,1,...,WA,WA3,301.647233,289.308785,193.246849,262.010594,53.46955,-2.583977,361332.0,397136.0
2,2018-08-29,OL6 6RJ,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,112.500,1,0,1,0,...,OL,OL6,197.639388,183.803044,196.658531,262.010594,53.48984,-2.074338,395165.0,399237.0
3,2018-06-15,M46 0TW,MANCHESTER,WIGAN,GREATER MANCHESTER,184.995,1,0,0,1,...,M,M46,286.373466,285.381745,193.246849,262.010594,53.52589,-2.495181,367270.0,403360.0
4,2018-09-28,M28 3XS,MANCHESTER,SALFORD,GREATER MANCHESTER,214.995,0,1,0,1,...,M,M28,286.373466,285.381745,253.098761,262.010594,53.51677,-2.411523,372810.0,402310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195234,2025-01-17,NN4 8SA,NORTHAMPTON,WEST NORTHAMPTONSHIRE,WEST NORTHAMPTONSHIRE,98.000,0,1,1,0,...,NN,NN4,336.261460,360.532742,406.948231,406.948231,52.22464,-0.916339,474118.0,259044.0
7195235,2025-01-24,NN16 0SE,KETTERING,NORTH NORTHAMPTONSHIRE,NORTH NORTHAMPTONSHIRE,120.000,0,1,1,0,...,NN,NN16,336.261460,309.328992,333.187457,333.187457,52.39974,-0.721861,487058.0,278736.0
7195236,2025-01-14,NN10 0NB,RUSHDEN,NORTH NORTHAMPTONSHIRE,NORTH NORTHAMPTONSHIRE,604.500,1,0,1,0,...,NN,NN10,336.261460,262.890159,333.187457,333.187457,52.28271,-0.591401,496187.0,265885.0
7195237,2025-01-10,NN4 8TE,NORTHAMPTON,WEST NORTHAMPTONSHIRE,WEST NORTHAMPTONSHIRE,133.650,0,1,1,0,...,NN,NN4,336.261460,360.532742,406.948231,406.948231,52.22892,-0.921594,473752.0,259515.0


In [21]:
print(loc_house_df['LONG'].min())
print(loc_house_df['LAT'].min())
print(loc_house_df['LONG'].max())
print(loc_house_df['LAT'].max())

-6.352711
49.89516
1.760167
55.79741


In [22]:
upper = loc_house_df['Price (Thousands)'].quantile(0.95)
df_no_outliers =  loc_house_df['Price (Thousands)'] <= upper
df_no_outliers.describe()

count     7195239
unique          2
top          True
freq      6835477
Name: Price (Thousands), dtype: object

In [23]:
df_no_outliers.to_csv('ETL_Final.csv', index=False)